# Simple Linear Regression
You have previously learned about simple linear regression models. In these models, what you try to do is fit a linear relationship between two variables. Let's refresh our memory with the example below. Here, we are trying to find a relationship between seniority and monthly income. The monthly income is shown in units of $1000 USD.

y = data["mpg"]
X_baseline = data[["weight"]]
baseline_model = sm.OLS(y, sm.add_constant(X_baseline))
baseline_results = baseline_model.fit()

print(baseline_results.summary())

The regression line:

fig, ax = plt.subplots()
data.plot.scatter(x="weight", y="mpg", label="Data points", ax=ax)
sm.graphics.abline_plot(model_results=baseline_results, label="Regression line", ax=ax, color="black")
ax.legend();

And the residuals:

fig, ax = plt.subplots()

ax.scatter(data["mpg"], baseline_results.resid)
ax.axhline(y=0, color="black")
ax.set_xlabel("weight")
ax.set_ylabel("residuals");

From our simple linear regression to a multiple linear regression. The process of building this model with StatsModels is very similar to the process of building our baseline simple regression model; this time we simply create an X variable containing multiple columns.

X_second = data[["weight", "model year"]]
X_second

second_model = sm.OLS(y, sm.add_constant(X_second))
second_results = second_model.fit()

print(second_results.summary())

Model Fit
sm.graphics.plot_fit(second_results, "weight")
plt.show()
We can also plot the fit for the other predictor, model year:

sm.graphics.plot_fit(second_results, "model year")
plt.show()

Partial Regression Plot
Then, instead of a basic scatter plot with a best-fit line (since our model is now higher-dimensional), we'll use two partial regression plots, one for each of our predictors.

fig = plt.figure(figsize=(15,5))
sm.graphics.plot_partregress_grid(second_results, exog_idx=["weight", "model year"], fig=fig)
plt.tight_layout()
plt.show()

Plotting Residuals
One approach to plotting residuals from a multiple regression model is to use the same approach as in simple regression, just plotting the value of the predictor on the x-axis vs. the model residuals on the y-axis.

fig, axes = plt.subplots(ncols=2, figsize=(15,5), sharey=True)

weight_ax = axes[0]
weight_ax.scatter(data["weight"], second_results.resid)
weight_ax.axhline(y=0, color="black")
weight_ax.set_xlabel("weight")
weight_ax.set_ylabel("residuals")

year_ax = axes[1]
year_ax.scatter(data["model year"], second_results.resid)
year_ax.axhline(y=0, color="black")
year_ax.set_xlabel("model year");

Plotting All Four at Once
If you are interested in creating all of these plots at once for a given predictor, StatsModels has a wrapper that can do this also:

fig = plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(second_results, "weight", fig=fig)
plt.show()

# Linear Regression Using scikit-learn
from sklearn.linear_model import LinearRegression

sklearn_third_model = LinearRegression()
sklearn_third_model.fit(X_all, y)

print(f"""
StatsModels R-Squared:    {third_results.rsquared}
scikit-learn R-Squared:   {sklearn_third_model.score(X_all, y)}

StatsModels intercept:     {third_results.params["const"]}
scikit-learn intercept:    {sklearn_third_model.intercept_}

StatsModels coefficients:  {third_results.params[1:].values}
scikit-learn coefficients: {sklearn_third_model.coef_}
""")

# Multiple Regression Model Evaluation
Multiple regression models, like simple regression models, can be evaluated using R-Squared (coefficient of determination) for measuring the amount of explained variance, and the F-statistic for determining statistical significance. You also may want to consider using other metrics, including adjusted R-Squared and error-based metrics such as MAE and RMSE

import pandas as pd
import statsmodels.api as sm
data = pd.read_csv("auto-mpg.csv")
data
y = data["mpg"]
X = data[["weight", "model year"]]
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()
results.fvalue, results.f_pvalue
*very small p-value, we can say that the model is statistically significant.*
results.rsquared
*This means that our model is explaining about 81% of the variance in mpg.*

Some limitations of R-Squared include : First, as we add more predictors, R-Squared is only going to increase. Second, "proportion of variance explained" may not be the best way to describe your model's performance. These drawbacks are addressed by adjusted R-Squared

results.rsquared_adj

Some issues with R-Squared can't be addressed with small tweaks like adjusted R-Squared. While R-Squared is a relative metric that compares the variance explained by the model to the variance explained by an intercept-only "baseline" model, most error-based metrics are absolute metrics that describe some form of average error.

## Mean Absolute Error
Mean absolute error (MAE) calculates the absolute value of each error before adding it to the sum. We can just use the .abs() method on the residuals series to do this:
results.resid.abs()
mae = results.resid.abs().sum() / len(y)
mae
*a lower MAE is better, and the smallest theoretically possible MAE is 0.*

## Root Mean Squared Error
Another popular error-based metric is root mean squared error. Root mean squared error (RMSE) calculates the squared value of each error, sums them, then takes the square root at the end.
results.resid ** 2
rmse = ((results.resid ** 2).sum() / len(y)) ** 0.5
rmse

# Calculating Metrics with Scikit-Learn
from sklearn.metrics import mean_absolute_error, mean_squared_error
mean_absolute_error(y, y_pred)
mean_squared_error(y, y_pred, squared=False)

# Dealing with Categorical Variables
Engineer a new feature, make, using the car name feature:

data["make"] = data["car name"].str.split().apply(lambda x: x[0])
data

data[["make", "origin"]].groupby("make").first().sort_values("origin")
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,5))

data.plot.scatter(x="origin", y="mpg", ax=ax1)
data.groupby("origin").mean('mpg').plot.bar(y='mpg', ax=ax2);

 A continuous variable is essentially always numeric, and a string variable is essentially always categorical.

## Transforming Categorical Variables with One-Hot Encoding
You can use the get_dummies function from pandas.

origin_df = data[["origin"]].copy()
origin_df.sample(10, random_state=1)

We can also do one-hot encoding on the entire DataFrame at once, just specifying the columns we consider to be categorical:

pd.get_dummies(data, columns=["origin", "make"], dtype=int

## Multiple Regression with One-Hot Encoded Variables
Let's go ahead and create a linear regression model with weight, model year, and origin.

y = data["mpg"]
X = data[["weight", "model year", "origin"]]
X
X = pd.get_dummies(X, columns=["origin"], drop_first=True, dtype=int)
X
import statsmodels.api as sm

model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

print(results.summary())

## One-Hot Encoding with Scikit-Learn
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop="first", sparse_output=False)

we typically use StatsModels to build multiple regression models, just like we did for simple regression models
Another library that can be used for linear regression is scikit-learn. This is mostly relevant for predictive modeling (machine learning) because it doesn't calculate p-values for coefficients
There are some problems with R-Squared (coefficient of determination) for evaluating multiple regression models
R-Squared will only ever increase as we add more features. Adjusted R-Squared accounts for the number of features and is a better metric for multiple regression
Proportion of variance explained is not intuitive for stakeholders. Error-based metrics such as mean absolute error (MAE) and root mean squared error (RMSE) are helpful tools to express how incorrect the model is in an average prediction
Now that we have the ability to use multiple features in our regression models, we can use categorical features
Categorical features must be preprocessed before they can be used in linear regression models
Specifically we used one-hot encoding to create dummy variables (1s or 0s) representing each category
In order to avoid the dummy variable trap we need to drop one of the dummy variable columns. Whichever column is dropped becomes the reference category, where all other category coefficients are compared to this category.